In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
import pandas as pd
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
img_height = 28
img_width = 28
batch_size = 2

model = keras.Sequential([
    keras.Input((28,28,1)),
    layers.Conv2D(16, 3, activation='relu'),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

In [3]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    't18_data/mnist_subfolders/',
    labels='inferred',
    label_mode='int', #categorical(ohe), binary
    #class_names=[],
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='training')

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    't18_data/mnist_subfolders/',
    labels='inferred',
    label_mode='int', #categorical(ohe), binary
    #class_names=[],
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.1,
    subset='validation')

Found 50 files belonging to 10 classes.
Using 45 files for training.
Found 50 files belonging to 10 classes.
Using 5 files for validation.


In [4]:
def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y

ds_train = ds_train.map(augment)

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])

model.fit(ds_train,epochs=10)

Epoch 1/10
23/23 [==============================] - 1s 5ms/step - loss: 39.5467 - accuracy: 0.0854
Epoch 2/10
23/23 [==============================] - 0s 2ms/step - loss: 2.4161 - accuracy: 0.8021
Epoch 3/10
23/23 [==============================] - 0s 2ms/step - loss: 0.0322 - accuracy: 0.9753
Epoch 4/10
23/23 [==============================] - 0s 3ms/step - loss: 3.8820e-05 - accuracy: 1.0000
Epoch 5/10
23/23 [==============================] - 0s 3ms/step - loss: 1.5877e-05 - accuracy: 1.0000
Epoch 6/10
23/23 [==============================] - 0s 3ms/step - loss: 2.1152e-05 - accuracy: 1.0000
Epoch 7/10
23/23 [==============================] - 0s 3ms/step - loss: 6.9297e-06 - accuracy: 1.0000
Epoch 8/10
23/23 [==============================] - 0s 3ms/step - loss: 1.7399e-05 - accuracy: 1.0000
Epoch 9/10
23/23 [==============================] - 0s 3ms/step - loss: 1.5443e-05 - accuracy: 1.0000
Epoch 10/10
23/23 [==============================] - 0s 3ms/step - loss: 1.4119e-05 - accurac

In [5]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range=(0.95,0.95),
    horizontal_flip=False,
    vertical_flip=False,
    data_format='channels_last',
    validation_split=0.0,
    dtype=tf.float32)

train_generator = datagen.flow_from_directory(
    't18_data/mnist_subfolders/',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='sparse',
    shuffle=True,
    subset='training',
    seed=123)

#custom loops
def training(): pass

for epoch in range(10):
    num_batches = 0
    
    for x, y in ds_train:
        num_batches += 1
        
        training()
        
        if num_batches == 25:
            break

Found 50 images belonging to 10 classes.


In [6]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])

model.fit(train_generator,epochs=10,steps_per_epoch=25)

Epoch 1/10
25/25 [==============================] - 0s 2ms/step - loss: 2.1400 - accuracy: 0.5989
Epoch 2/10
25/25 [==============================] - 0s 2ms/step - loss: 1.1400 - accuracy: 0.8317
Epoch 3/10
25/25 [==============================] - 0s 3ms/step - loss: 0.2364 - accuracy: 0.9488
Epoch 4/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0833 - accuracy: 0.9752
Epoch 5/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0427 - accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0053 - accuracy: 1.0000
Epoch 8/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0059 - accuracy: 1.0000
Epoch 9/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 10/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 1.0000


In [7]:
directory = 't18_data/mnist_images_csv/'
df = pd.read_csv(directory+'train.csv')

In [8]:
file_paths = df['file_name'].values
file_paths[:5]

array(['0_1.jpg', '0_2.jpg', '0_3.jpg', '0_4.jpg', '0_5.jpg'],
      dtype=object)

In [9]:
labels = df['label'].values
labels[:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [10]:
ds_train2 = tf.data.Dataset.from_tensor_slices((file_paths,labels))

def read_image(image_file, label):
    image = tf.io.read_file(directory + image_file)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)
    return image, label

ds_train2 = ds_train2.map(read_image).batch(2)

In [11]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])

model.fit(ds_train2,epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 3ms/step - loss: 1.8624 - accuracy: 0.7097
Epoch 2/10
25/25 [==============================] - 0s 2ms/step - loss: 0.4857 - accuracy: 0.8261
Epoch 3/10
25/25 [==============================] - 0s 2ms/step - loss: 0.4358 - accuracy: 0.9235
Epoch 4/10
25/25 [==============================] - 0s 2ms/step - loss: 0.1655 - accuracy: 0.9960
Epoch 5/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0714 - accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0344 - accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 1.0000
Epoch 8/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 9/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0094 - accuracy: 1.0000
Epoch 10/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0072 - accuracy: 1.0000


In [12]:
directory2 = 't18_data/mnist_images_only/'
ds_train3 = tf.data.Dataset.list_files(str(pathlib.Path(directory2+'*.jpg')))

def process_path(file_path):
    image = tf.io.read_file(file_path)
    image = tf.image.decode_jpeg(image, channels=1)
    label = tf.strings.split(file_path, '\\')
    label = tf.strings.substr(label, pos=0, len=1)[2]
    label = tf.strings.to_number(label, out_type=tf.int64)
    return image, label
    
ds_train3 = ds_train3.map(process_path).batch(batch_size)

In [13]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(),
    metrics=['accuracy'])

model.fit(ds_train3,epochs=10)

Epoch 1/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/10
25/25 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0000e+0